In [38]:
from transformers import DistilBertTokenizer, CamembertTokenizer, BertTokenizer, DistilBertModel, CamembertModel, BertModel, XLMTokenizer, XLMModel
import torch
import pandas as pd 
import ast
import matplotlib.pyplot as plt
from dataset2 import MWEDataset
from torch.utils.data import DataLoader
from transformer import BertMWE

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
bert_model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

train_dataset = MWEDataset(f"train_BIGO.csv", tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16)

batch = train_loader.__iter__().__next__()

model = BertMWE(4, bert_model, device)

In [41]:
from tqdm import tqdm

df = pd.read_csv("test_BIGO.csv", sep="\t", 
                              converters={"token_list": ast.literal_eval, 
                                          "lemmas": ast.literal_eval, 
                                          'labels': ast.literal_eval})

model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

count = 
for idx, row in tqdm(df.iterrows()): 
    tokens = row.token_list

    sentence = " ".join(tokens)
    tokens_bert = tokenizer.tokenize(sentence)

    encoding = tokenizer.encode_plus(tokens, add_special_tokens=True, 
                                max_length=512, padding='max_length', 
                                return_attention_mask=True, 
                                return_tensors='pt', truncation=True)
    input_ids, attention_mask = encoding['input_ids'], encoding['attention_mask']

    embedding = model(input_ids=input_ids, attention_mask=attention_mask)[0]

    if len(tokens_bert)>512 :
        final_emb = embedding
        break

1601it [20:06,  1.33it/s]


In [43]:
final_emb.shape

torch.Size([1, 512, 768])

In [58]:
id = torch.load("index_list_test.pt")
input1 = torch.load("input_id_2000.pt")
input2 = torch.load("input_id_4000.pt")
input3 = torch.load("input_id_6000.pt")
input4 = torch.load("input_id_8000.pt")
input5 = torch.load("input_id_10000.pt")
input6 = torch.load("input_id_12000.pt")
input7 = torch.load("input_id_14000.pt")
input8 = torch.load("input_id_fin.pt")

input = torch.cat((input1, input2, input3, input4, input5, input6, input7, input8), dim=0)

In [60]:
tensor = torch.load("embeddings/test/embeddings_tensor_test_IGO_2000.pt")
len(tensor)

1846

In [59]:
len(id), len(input)

(13398, 13398)

In [50]:
test = torch.load('embeddings/test/embeddings_tensor_test_BIGO_2000.pt')

for i in range(len(test)) :
    if torch.equal(test[i], final_emb[0]) :
        print(i)

In [31]:
import torch
import torch.nn.functional as F

def viterbi(emission_matrices, initial_proba, transition_proba):
    B, N, T = emission_matrices.shape

    delta = torch.zeros(B, N, T)
    psi = torch.zeros(B, N, T)

    # Initialisation
    delta[:, :, 0] = emission_matrices[:, :, 0] * initial_proba.view(1, -1)

    # Calcul des différentes valeurs de delta
    for t in range(1, T):
        for j in range(N):
            liste = [
                delta[b, i, t-1] * transition_proba[i, j] * emission_matrices[b, j, t]
                for b in range(B)
                for i in range(N)
            ]
            liste = torch.tensor(liste).view(B, N)
            delta[:, j, t], psi[:, j, t] = liste.max(dim=1)

    # Détermination du meilleur chemin
    z_T = delta[:, :, T - 1].argmax(dim=1)
    z = torch.zeros(B, T, dtype=torch.long)
    z[:, T - 1] = z_T

    for t in range(T - 2, -1, -1):
        z[:, t] = psi[torch.arange(B), z[:, t + 1], t + 1]

    return z

# Example usage
initial_proba = torch.tensor([0.25, 0.25, 0.25, 0.25])
transition_proba = torch.rand(4, 4)
transition_proba = F.softmax(transition_proba, dim=1)

input_seq = ['le', 'chat', 'adore', 'le', 'lait', '.']
emission_proba = torch.rand(len(input_seq), 4).T
emission_proba[:, 0] = torch.tensor([1, 0, 0, 0])
emission_proba[:, 1] = torch.tensor([0, 1, 0, 0])
emission_proba[:, 2] = torch.tensor([0, 0, 0, 1])
emission_proba[:, 3] = torch.tensor([-1, 0, 1, 0])
emission_proba[:, 4] = torch.tensor([0, 0, 0, 1])
emission_proba[:, 5] = torch.tensor([1, 0, 0, 0])

emission_proba = emission_proba.unsqueeze(0)
emission_proba = torch.cat((emission_proba, emission_proba), dim=0)
emission_proba = F.softmax(emission_proba, dim=1)

print(emission_proba.shape, initial_proba.shape, transition_proba.shape)
print(viterbi(emission_proba, initial_proba, transition_proba))



torch.Size([2, 4, 6]) torch.Size([4]) torch.Size([4, 4])
tensor([[0, 1, 3, 2, 3, 0],
        [0, 1, 3, 2, 3, 0]])


In [14]:
def viterbi(emission_matrix, initial_proba, transition_proba) :
    shape = emission_matrix.shape

    delta = torch.zeros(shape)
    psi = torch.zeros(shape)

    # Initialisation
    delta[:, 0] = emission_matrix[:, 0] * initial_proba

    # Calcul des différentes valeurs de delta
    for t in range(0, shape[1] - 1) :
        for j in range(shape[0]) :
            liste = [ delta[i, t] * transition_proba[i, j] * emission_matrix[j, t + 1] for i in range(shape[0])]
            delta[j, t + 1], psi[j, t + 1] = max(liste), torch.argmax(torch.tensor(liste))

    # Détermination du meilleur chemin 
    z_T = torch.argmax(torch.tensor([delta[i, shape[1]-1] for i in range(shape[0])])).long()
    z = torch.zeros(shape[1]).long()
    z[shape[1]-1] = z_T
    for t in range(shape[1]-2, 0, -1) :
        z[t] = psi[z[t+1], t+1]
    
    return z

In [33]:
from viterbi import get_initial_proba, get_transition_proba
import torch.nn.functional as F

initial_proba = torch.tensor(get_initial_proba("train_BIGO.csv"))
transition_proba = torch.tensor(get_transition_proba("train_BIGO.csv"))


input = ['le', 'chat', 'adore', 'le', "lait", '.']
emission_proba = torch.rand(len(input), 4).T
emission_proba[:, 0] = torch.tensor([1, -1, -1, 2])
emission_proba[:, 1] = torch.tensor([-1, 1, -1, -1])
emission_proba[:, 2] = torch.tensor([-1, -1, -1, 1])
emission_proba[:, 3] = torch.tensor([-1, -1, 1, -1])
emission_proba[:, 4] = torch.tensor([1, -1, -1, -1])
emission_proba[:, 5] = torch.tensor([1, -1, -1, 2])

emission_proba = emission_proba.unsqueeze(0)
emission_proba = torch.cat((emission_proba, emission_proba), dim=0)
emission_proba = F.softmax(emission_proba, dim=2)
# emission_proba = F.softmax(emission_proba, dim=1)
print(emission_proba.shape, initial_proba.shape, transition_proba.shape)

print(viterbi(emission_proba, initial_proba, transition_proba))

torch.Size([2, 4, 6]) torch.Size([4]) torch.Size([4, 4])
tensor([[0, 1, 3, 2, 0, 0],
        [0, 1, 3, 2, 0, 0]])


In [2]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
cam = CamembertTokenizer.from_pretrained('camembert-base')
bert = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
xlm = XLMTokenizer.from_pretrained('xlm-mlm-enfr-1024')

# dataset = MWEDataset('test_BIGO.csv', tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
max_tok = 0
max_ind = 0
max_lab = 0
for i in range(len(dataset)) :
    _, _, tokens, indices, labels = dataset[i]
    if len(tokens) > max_tok :
        max_tok = len(tokens)
    if len(indices) > max_ind :
        max_ind = len(indices)
    if len(labels) > max_lab :
        max_lab = len(labels)


In [15]:
max_tok, max_ind, max_lab

(400, 580, 400)

In [17]:
df = pd.read_csv('train_BIGO.csv', sep="\t", 
                              converters={"token_list": ast.literal_eval, 
                                          "lemmas": ast.literal_eval, 
                                          'labels': ast.literal_eval})

tokens = df.token_list.iloc[0]
print(tokens)
lemmas = df.lemmas.iloc[0]
print(len(lemmas))

enc = cam.encode_plus(lemmas)
print(enc["input_ids"])
print(len(enc["input_ids"]))

sentence = " ".join(lemmas)
encoding = cam.tokenize(sentence)
print(encoding)

labels = df.labels.iloc[0]
print(labels)



["l'", 'ananas', 'flambé', 'au', 'à', 'le', 'rhum', 'ne', 'venait', 'pas', "d'", 'une', 'boite', 'de', 'conserve', '.']
16
[5, 185, 24852, 3, 862, 169, 185, 3, 324, 13319, 2475, 234, 59, 3, 234, 3, 9, 6]
18
['▁le', '▁', 'ananas', '▁fla', 'mber', '▁_', '▁à', '▁le', '▁rhum', '▁ne', '▁venir', '▁pas', '▁de', '▁un', '▁boîte', '▁de', '▁conserve', '▁', '.']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [114]:
token = 'on'
token in ('ce', 'on')

True

In [226]:
def mapping_tokens(tokens : list[str]): 
            list_mapping, counter, is_dash, is_parenthesis, is_digits, count_dash = [0], 0, False, False, False, 0
            for idx, token in enumerate(tokens):
                if not(token.startswith("##")):
                    if token == '[UNK]' :
                        token = "'"
                    elif token == "hui" and tokens[idx-1] == "'":
                        continue
                    
                    elif "'" in token: 
                        is_dash = False
                    elif is_dash:
                        if token in ("ce", "on"): 
                            counter += 1
                            is_dash = False
                        elif token == "-":
                            count_dash += 1
                            is_dash=True
                        else:
                            is_dash=False
                            count_dash = 0

                        if count_dash == 2 :
                            counter +=1 
                            is_dash = False
                            count_dash = 0
                            list_mapping[-1] += 1

                    elif "-" in token:
                        if idx == 0 :
                            counter += 1
                        elif tokens[idx-1] == "a":
                            counter += 1
                            is_dash = True
                            count_dash += 1
                        else : 
                            is_dash = True
                            count_dash += 1
                            
                    elif token in ("(", ")"): 
                        if token == "(": 
                            token_after = tokens[idx+1]
                            if len(token_after) >= 2: 
                                counter += 1
                            elif token_after.isdigit():
                                counter += 1
                            else : 
                                is_parenthesis = True
                        else: 
                            token_before = tokens[idx-1]
                            if len(token_before) >= 2: 
                                counter += 1
                            is_parenthesis = False
                    elif is_parenthesis: 
                        is_parenthesis=False

                    elif token.isdigit():
                        if not is_digits:
                            counter += 1
                            is_digits = True
                    elif is_digits:
                        if not (token == "," and tokens[idx+1].isdigit()) :
                            is_digits = False
                            counter += 1

                    elif token == '.' :
                        if tokens[idx-1] in ("etc", "cf", '.'):
                            continue

                        else :
                            counter += 1

                    else :
                        counter += 1
                list_mapping.append(counter)
            return list_mapping

tokens_test = ['Le', 'cha', "##t", "d", "''", "élevage", "traité", "(", "e", ")", "test", "(", "vieux", "monsieur", ")", 'su', '##ffi', '##xe', '-', '-', 'villa']
tokens_test = ["Arb", '##re', '7', '736', "890", "6", 'su', 'test', '7', '786', "fin"]
tokens_test = ["Saint", "-", "germain", 'su', '##ffi', '##xe', '-', '-', 'villa']
list_mapping = mapping_tokens(tokens_test)

In [183]:
list_mapping

[0, 1, 1, 1, 2, 2, 2, 3, 3, 4]

In [4]:
labels = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [125]:
from dataset2 import MWEDataset

dataset = MWEDataset('train_BIGO.csv', tokenizer=tokenizer)

def create_mapped_lists(second_tokens, mapping):
    result = []

    for i in range(len(mapping)) :
        if i > 0 :
            if len(result)== 0 :
                result.append([second_tokens[i-1]])
            elif mapping[i] == mapping[i-1] :
                result[-1].append(second_tokens[i-1])
            else :
                result.append([second_tokens[i-1]])
    
    return result

In [225]:
count = 0
lab_count = 0
nb_sentences = 0
for i in range(len(dataset)) :
    nb_sentences += 1
    tokens, bert_tokens, labels = dataset[i]
    list_mapping = mapping_tokens(bert_tokens)
    if max(list_mapping) != len(labels) - 2 :
        results = create_mapped_lists(bert_tokens, list_mapping)
        print(results)
        print(tokens)
        count += 1
        if count == 4 :
            break

[['Il'], ['était'], ['construit'], ['par'], ['les'], ['ateliers'], ['La'], ['Prairie'], ['à'], ['L', "'"], ['Isle', '-', 'd', "'"], ['Es', '##pagna', '##c'], ['('], ['Charente'], [')'], [','], ['spécial', '##isés'], ['dans'], ['la'], ['fabrication'], ['de'], ['matériel'], ['de'], ['camping'], [';']]
['Il', 'était', 'construit', 'par', 'les', 'ateliers', 'La', 'Prairie', 'à', "L'", "Isle-d'Espagnac", '(', 'Charente', ')', ',', 'spécialisés', 'dans', 'la', 'fabrication', 'de', 'matériel', 'de', 'camping', ';']
[['Les'], ['en', '##co', '##ches'], ['ou'], ['c', '##rans'], ['sur'], ['les'], ['plan', '-', 'films'], ['permettent'], ['d', "'"], ['ide', '##nti', '##fier'], ['les'], ['marques'], ['et'], ['types'], ['d', "'"], ['é', '##mul', '##sion'], ['('], ['sens', '##ibil', '##ité'], [','], ['co', '##uc', '##leur'], [','], ['noir'], ['&'], ['blanc'], [','], ['in', '##versi', '##ble'], [')']]
['Les', 'encoches', 'ou', 'crans', 'sur', 'les', 'plan-films', 'permettent', "d'", 'identifier', 'les'

In [227]:
count = 0
lab_count = 0
nb_sentences = 0
for i in range(len(dataset)) :
    nb_sentences += 1
    tokens, bert_tokens, labels = dataset[i]
    list_mapping = mapping_tokens(bert_tokens)
    if max(list_mapping) != len(labels) -2 :
        count += 1
        for elt in labels :
            if elt != 0 and elt != -100 :
                lab_count += 1
                break

In [228]:
count, nb_sentences, lab_count

(1142, 14540, 304)

In [2]:
list_mapping

[0, 1, 2, 2, 3, 3, 4, 5, 5, 5, 5, 6, 7, 8, 9, 10]

In [67]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def mean_embed(list_mapping : list[int], embeddings : torch.Tensor):
            current_map, list_current_embed, embeddings_final = None, None, None
            for idx, mapping in enumerate(list_mapping): 

                if idx==0: 
                    current_map = mapping
                    list_current_embed = [embeddings[idx, :]]

                elif idx==len(list_mapping)-1:
                    if current_map == mapping:
                        list_current_embed.append(embeddings[idx, :])
                    else: 
                        mean_embeddings = None
                        for idx_embed, embed in enumerate(list_current_embed): 
                                if idx_embed==0: 
                                    mean_embeddings = embed
                                else : 
                                    mean_embeddings += embed
                        new_line = mean_embeddings/len(list_current_embed)
                        embeddings_final = torch.cat((embeddings_final, new_line.detach().unsqueeze(0).to(device)), dim=0)
                        list_current_embed = [embeddings[idx, :]]
                    mean_embeddings = None
                    for idx_embed, embed in enumerate(list_current_embed): 
                            if idx_embed==0: 
                                mean_embeddings = embed
                            else : 
                                mean_embeddings += embed 
                    new_line = mean_embeddings/len(list_current_embed)
                    embeddings_final = torch.cat((embeddings_final, new_line.detach().unsqueeze(0).to(device), embeddings[idx+1:,:].to(device)), dim=0)

                elif current_map != mapping:
                    mean_embeddings = None
                    for idx_embed, embed in enumerate(list_current_embed): 
                            if idx_embed==0: 
                                mean_embeddings = embed
                            else : 
                                mean_embeddings += embed
                    if embeddings_final is not None:
                        new_line = mean_embeddings/len(list_current_embed)
                        embeddings_final = torch.cat((embeddings_final, new_line.detach().unsqueeze(0).to(device)), dim=0)
                    else: 
                        new_line = mean_embeddings/len(list_current_embed)
                        embeddings_final = torch.tensor(new_line.detach().to(device)).unsqueeze(0)

                    list_current_embed = [embeddings[idx, :]]
                    current_map = mapping

                else : 
                    list_current_embed.append(embeddings[idx, :])

            return embeddings_final

embeddings_tensor = torch.tensor([[-1, -1, -1], [1, 2, 3], [1, 2, 3], [-1, -2, -3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15], [-13, -14, -15], [16, 17, 18], [-10, -10, -10],  [-10, -10, -10],  [-10, -10, -10]])
mean_embed(list_mapping, embeddings_tensor)

tensor([[ -1,  -1,  -1],
        [  1,   2,   3],
        [  1,   2,   3],
        [ -1,  -2,  -3],
        [  4,   5,   6],
        [  7,   8,   9],
        [ 10,  11,  12],
        [ 13,  14,  15],
        [-13, -14, -15],
        [ 16,  17,  18],
        [-10, -10, -10],
        [-10, -10, -10],
        [-10, -10, -10]])


C:\Users\alexa\AppData\Local\Temp\ipykernel_6540\4261732794.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_final = torch.tensor(new_line.detach().to(device)).unsqueeze(0)


tensor([[ -1.,  -1.,  -1.],
        [  1.,   2.,   3.],
        [  0.,   0.,   0.],
        [  4.,   5.,   6.],
        [  7.,   8.,   9.],
        [ 10.,  11.,  12.],
        [  0.,   0.,   0.],
        [ 16.,  17.,  18.],
        [-10., -10., -10.],
        [-10., -10., -10.],
        [-10., -10., -10.]], device='cuda:0')

In [22]:
[[1,2,3]].squeeze(0)

AttributeError: 'list' object has no attribute 'squeeze'

In [19]:
for index, row in df.iterrows():
    print(row["token_list"])
    break


["l'", 'ananas', 'flambé', 'au', 'à', 'le', 'rhum', 'ne', 'venait', 'pas', "d'", 'une', 'boite', 'de', 'conserve', '.']


In [21]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from create_embeddings import MWEDataset
from transformer import CamembertMWE
from transformers import CamembertTokenizer, CamembertModel
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
camembert_model = CamembertModel.from_pretrained('camembert-base')

learning_rate = 5e-5
epochs = 1
batch_size = 16

print("Loading datasets...")
test_dataset = MWEDataset("test_BIGO.csv", tokenizer)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = CamembertMWE(4, camembert_model, "cpu")

for param in model.bert.base_model.parameters():
    param.requires_grad = False

optimizer = AdamW(model.parameters(), lr=learning_rate)
loss = CrossEntropyLoss(reduction='none')

batch = next(iter(test_loader))
logits = model(batch[0], batch[1])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading datasets...


RuntimeError: each element in list of batch should be of equal size

In [22]:
_, preds = torch.max(logits, dim=2)

NameError: name 'logits' is not defined

In [23]:
preds

NameError: name 'preds' is not defined

In [24]:
def calculate_scores_by_class(y_true, y_pred):
    # Filter out cases where y_true is -100
    valid_indices = [i for i, label in enumerate(y_true) if label != -100]
    y_true_valid = np.array([y_true[i] for i in valid_indices])
    y_pred_valid = np.array([y_pred[i] for i in valid_indices])

    # Calculate F1 score by class
    f1_scores = f1_score(y_true_valid, y_pred_valid, average=None, labels=np.unique(y_true_valid))

    # Calculate accuracy score by class
    accuracy_scores = accuracy_score(y_true_valid, y_pred_valid, normalize=False)
    total_valid_samples = len(y_true_valid)
    accuracy_scores /= total_valid_samples

    return f1_scores, accuracy_scores

# Example usage:
y_true = [-100, 0, 0, 0, 1, 2, 3, 0, 0, 0, -100]
y_pred = [1, 3, 2, 0, 0, 0, 1, 3, 0, 0, 0]

f1_scores, accuracy_scores = calculate_scores_by_class(y_true, y_pred)

In [34]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def calculate_scores_by_class(y_true, y_pred):
    # Filter out cases where y_true is -100
    valid_indices = [i for i, label in enumerate(y_true) if label != -100]
    y_true_valid = np.array([y_true[i] for i in valid_indices])
    y_pred_valid = np.array([y_pred[i] for i in valid_indices])

    # Calculate F1 score by class
    f1_scores = f1_score(y_true_valid, y_pred_valid, average=None, labels=np.unique(y_true_valid))

    return f1_scores

# Example usage:
y_true = [-100, 0, 0, 0, 1, 2, 3, 0, 0, 0, -100]
y_pred = [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]

f1_scores = calculate_scores_by_class(y_true, y_pred)


In [35]:
f1_scores

array([0.92307692, 0.66666667, 0.        , 0.        ])

In [26]:
df = pd.read_csv('train_BIGO.csv', sep="\t", 
                              converters={"token_list": ast.literal_eval, 
                                          "lemmas": ast.literal_eval, 
                                          'labels': ast.literal_eval})

labels = df.labels

In [27]:
O = 0
B = 0
I = 0
G = 0
for label in labels :
    for l in label :
        if l == 0 :
            O += 1
        elif l == 1 :
            B += 1
        elif l == 2 :
            I += 1
        elif l == 3 :
            G += 1

In [28]:
print(O, B, I, G)

352744 3865 5003 2802


In [29]:
somme = O + I + G
100-(O*100/somme), 100-(B*100/somme), 100-(I*100/somme), 100-(G*100/somme)

(2.1647543052400664, 98.92802365281834, 98.61239387711518, 99.22285181764475)

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data):
        self.labels = data['labels']
        self.inputs = data['inputs']
        self.tests = data['tests']

    def __len__(self):
        return len(self.labels)  # Assuming all lists have the same length

    def __getitem__(self, idx):
        label = torch.tensor(self.labels[idx])
        input_data = torch.tensor(self.inputs[idx])
        test_data = torch.tensor(self.tests[idx])
        return label, input_data, test_data

# Example data
data = {
    'labels': [[1, 2], [1, 2]],
    'inputs': [[0.1, 0.2, .3], [0.3, 0.4, 0.5], [0.6, 0.7, 0.8]],
    'tests': [[0.01, 0.02, 0.03], [0.04, 0.05, .3], [0.06, 0.07, 0.08]]
}

# Create dataset and dataloader
my_dataset = MyDataset(data)
batch_size = 2
my_dataloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True)

# Iterate through batches
for batch in my_dataloader:
    labels, inputs, tests = batch
    print("Labels:", labels)
    print("Inputs:", inputs)
    print("Tests:", tests)
    print("Shapes - Labels:", labels.shape, "Inputs:", inputs.shape, "Tests:", tests.shape)
    print("\n")


Labels: tensor([[1, 2],
        [1, 2]])
Inputs: tensor([[0.1000, 0.2000, 0.3000],
        [0.3000, 0.4000, 0.5000]])
Tests: tensor([[0.0100, 0.0200, 0.0300],
        [0.0400, 0.0500, 0.3000]])
Shapes - Labels: torch.Size([2, 2]) Inputs: torch.Size([2, 3]) Tests: torch.Size([2, 3])


